# * Ex-T Revenue & Subs

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [3]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
         , REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    --WHERE FLAG = 1
    GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), REMARK, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-06-26, 14:47:20

   -> Summany DataFrame : 441 rows, 18 columns

TDMDBPR : Disconnected


In [4]:
y2024_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH'].str.contains('2024')].copy()
# y2024_df['TM_KEY_YR'] = y2024_df['TM_KEY_MTH'].str.split().str[0]
y2024_df = y2024_df.groupby(['TM_KEY_YR', 'REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
y2024_df = y2024_df.fillna(0).sort_values(by=['TM_KEY_YR', 'REMARK', 'METRIC_CD']).reset_index()

mod_col_list = y2024_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    y2024_df[col] = y2024_df[col].apply(lambda x: format(x, ',.0f'))

y2024_df
# print(f'\n{y2024_df.to_string(max_cols=100)}')

,TM_KEY_YR,REMARK,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,2024,jutar11,TB1R000100,Prepay Revenue,2025-02-13 20:20:49,20240101,20241231,0,0,"27,855,545,957","27,855,124,991","27,855,124,991",0,0,4,176,64416
1,2024,jutar11,TB1R000101,Prepay New Revenue,2025-02-13 20:20:49,20240101,20241231,0,0,"5,978,086,564","5,978,086,564","5,978,086,564",0,0,4,176,64416
2,2024,jutar11,TB1R000102,Prepay Existing Revenue,2025-02-13 20:20:49,20240101,20241231,0,0,"21,877,459,392","21,877,038,426","21,877,038,426",0,0,4,176,64416
3,2024,jutar11,TB1S000500,Prepay Usage Subs,2025-02-13 20:46:40,20240101,20241231,0,0,"6,451,575,497","6,451,482,909","6,451,482,909",0,0,4,176,64240
4,2024,jutar11,TB2R000100,Postpaid Revenue,2025-02-13 20:20:49,20240102,20240630,0,"22,148,096,907","21,957,461,500","21,957,332,206","21,957,332,206",0,0,5,177,10620
5,2024,jutar11,TB2R000101,Postpaid New Revenue,2025-02-13 20:20:49,20240102,20241231,0,0,"4,372,937,033","4,372,938,478","4,372,945,380",0,0,4,176,20901
6,2024,jutar11,TB2R000102,Postpaid Existing Revenue,2025-02-13 20:20:49,20240102,20241231,0,"39,968,004,424","39,611,123,974","39,610,910,135","39,610,910,135",0,0,5,177,21240
7,2024,jutar11,TB2R010100,Postpaid Revenue B2C,2025-02-13 20:20:49,20240102,20240630,0,"19,688,902,597","19,654,753,585","19,654,632,823","19,654,632,823",0,0,5,177,10596
8,2024,jutar11,TB2R020100,Postpaid Revenue B2B,2025-02-13 20:20:49,20240102,20240630,0,"2,459,194,310","2,302,707,915","2,302,699,383","2,302,699,383",0,0,5,177,10620
9,2024,jutar11,TB2R020100SMB,Postpaid Revenue SMB : TMH,2025-02-13 20:20:49,20240102,20241231,0,"1,028,929,122","1,017,979,576","1,018,100,806","1,018,123,969",0,0,5,177,20316


In [5]:
y2025_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH'].str.contains('2025')].copy()
y2025_df = y2025_df.groupby(['TM_KEY_YR', 'REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
y2025_df = y2025_df.fillna(0).sort_values(by=['TM_KEY_YR', 'REMARK', 'METRIC_CD']).reset_index()

mod_col_list = y2025_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    y2025_df[col] = y2025_df[col].apply(lambda x: format(x, ',.0f'))

y2025_df
# print(f'\n{y2025_df.to_string(max_cols=100)}')

,TM_KEY_YR,REMARK,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,2025,jutar11,TB1R000101,Prepay New Revenue,2025-04-08 04:49:17,20250101,20250405,0,0,"856,187,185","856,187,185","856,187,185",0,0,4,176,16720
1,2025,jutar11,TB1R000102,Prepay Existing Revenue,2025-04-08 04:51:27,20250101,20250405,0,0,"6,894,731,850","6,894,692,394","6,894,692,394",0,0,4,176,16720
2,2025,jutar11,TB1S000500,Prepay Usage Subs,2025-06-25 04:50:06,20250101,20250623,0,0,"3,013,368,521","3,013,359,949","3,013,359,949",0,0,4,176,30624
3,2025,jutar11,TB2R000101,Postpaid New Revenue,2025-04-04 04:21:38,20250102,20250402,0,0,"264,603,956","264,628,338","264,631,242",0,0,4,176,5147
4,2025,jutar11,TB2R000102,Postpaid Existing Revenue,2025-04-04 04:21:44,20250102,20250402,0,"11,521,837,139","11,464,739,285","11,464,730,525","11,464,730,525",0,0,5,177,5310
5,2025,jutar11,TB2R020100SMB,Postpaid Revenue SMB : TMH,2025-04-04 04:22:06,20250102,20250402,0,"284,643,739","283,181,365","283,189,663","283,189,663",0,0,5,177,5144
6,2025,jutar11,TB2S000500,Postpaid Active Subs,2025-06-25 05:03:02,20250101,20250624,0,"1,784,111,252","1,707,401,353","1,707,413,037","1,707,414,386",0,0,5,177,30971
7,2025,jutar11,TB2S010500,Postpaid Active Subs B2C,2025-06-25 05:04:02,20250101,20250624,0,"1,576,443,180","1,513,070,406","1,513,081,221","1,513,082,534",0,0,5,177,30971
8,2025,jutar11,TB2S020400,Postpaid Active Subs B2B,2025-06-25 05:05:03,20250101,20250624,0,"207,668,072","194,481,721","195,875,988","195,947,198",0,0,5,177,30554
9,2025,salisa,CAP00006,EBITDA Margin,2025-06-11 11:37:54,20250101,20250531,0,0,"70,103","557,630",0,0,0,2,72,10872


In [6]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

In [7]:
# ''' Manual Month Period '''

# curr_mth = 000000
# prev_mth = 000000

In [8]:
''' Current Month Revenue '''

rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB3R000100,Broadband Revenue,2025-06-25 05:03:35,20250602,20250622,0,0,"1,267,005,327","1,267,005,327","1,267,005,327",0,0,4,176,1232
1,TB4R000100,TVS Revenue,2025-06-25 05:08:44,20250602,20250622,0,0,"145,610,494","145,784,446","145,780,407",0,0,4,176,1232


In [9]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = prev_mth
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-02-13 20:20:49,20240101,20241231,0,0,"2,321,295,496","2,321,260,416","2,321,260,416",0,0,4,176,64416
1,TB2R000100,Postpaid Revenue,2025-02-13 20:20:49,20240102,20240630,0,"3,691,349,485","3,659,576,917","3,659,555,368","3,659,555,368",0,0,5,177,10620
2,TB2R010100,Postpaid Revenue B2C,2025-02-13 20:20:49,20240102,20240630,0,"3,281,483,766","3,275,792,264","3,275,772,137","3,275,772,137",0,0,5,177,10596
3,TB2R020100,Postpaid Revenue B2B,2025-02-13 20:20:49,20240102,20240630,0,"409,865,718","383,784,653","383,783,231","383,783,231",0,0,5,177,10620
4,TB3R000100,Broadband Revenue,2025-06-03 05:04:01,20240102,20250528,0,0,"1,580,258,212","1,580,168,809","1,580,144,056",0,0,4,176,26928
5,TB4R000100,TVS Revenue,2025-06-06 05:12:24,20240102,20250531,0,0,"323,314,506","318,363,647","321,922,202",0,0,5,1103,38894


In [10]:
''' Current Month Subscription '''

sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-06-25 04:50:06,20250601,20250623,0,0,"386,261,764","386,261,764","386,261,764",0,0,4,176,4048
1,TB2S000500,Postpaid Active Subs,2025-06-25 05:03:02,20250601,20250624,0,"245,415,330","245,404,719","245,405,362","245,405,444",0,0,5,177,4248
2,TB2S010500,Postpaid Active Subs B2C,2025-06-25 05:04:02,20250601,20250624,0,"216,876,932","216,872,943","216,873,585","216,873,667",0,0,5,177,4248
3,TB2S020400,Postpaid Active Subs B2B,2025-06-25 05:05:03,20250601,20250624,0,"28,538,398","28,531,776","28,531,777","28,531,777",0,0,5,177,4248
4,TB3S000500,Broadband Active Subs,2025-06-25 04:55:23,20250601,20250624,0,0,"75,349,229","75,349,229","75,349,229",0,0,4,176,4224
5,TB4S000500,TVS Active Subs,2025-06-25 04:55:38,20250601,20250624,0,0,"26,942,619","26,942,619","26,942,619",0,0,4,176,4224


In [11]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = prev_mth
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-06-10 04:50:49,20240101,20250531,0,0,"534,040,133","534,034,182","534,034,182",0,0,4,176,90816
1,TB2S000500,Postpaid Active Subs,2025-06-10 04:30:04,20240101,20250531,0,"304,825,468","294,741,524","294,953,073","294,994,064",0,0,5,177,91505
2,TB2S010500,Postpaid Active Subs B2C,2025-06-10 04:30:48,20240101,20250531,0,"270,420,437","263,582,457","263,778,444","263,818,529",0,0,5,177,91505
3,TB2S020400,Postpaid Active Subs B2B,2025-06-10 04:31:32,20240101,20250531,0,"34,405,032","31,219,174","31,415,512","31,433,338",0,0,5,177,91085
4,TB3S000500,Broadband Active Subs,2025-06-01 04:55:22,20240101,20250531,0,0,"91,955,886","92,178,648","92,178,640",0,0,4,176,89376
5,TB4S000500,TVS Active Subs,2025-06-01 04:55:40,20240101,20250531,0,0,"36,539,927","36,526,837","36,526,837",0,0,4,176,90288


In [12]:
# ''' METRIC Summary '''

# src_t_rev_mth_df = chk_src_df.copy()
# src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')

In [13]:
# ''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
# src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [14]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-06-26, 14:47:24



   -> Daily DataFrame : 108 rows, 12 columns

TDMDBPR : Disconnected


In [15]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 108 rows, 12 columns


In [16]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250301
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250601


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT


In [17]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250301
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250601


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT


In [18]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250301
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250601


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250602,2025-06-25 05:03:35,0,0,"219,573,994","219,573,994","219,573,994",0,0
1,salisa,TB3R000100,Broadband Revenue,20250606,2025-06-25 05:03:35,0,0,"146,409,035","146,409,035","146,409,035",0,0
2,salisa,TB3R000100,Broadband Revenue,20250610,2025-06-25 05:03:35,0,0,"232,115,357","232,115,357","232,115,357",0,0
3,salisa,TB3R000100,Broadband Revenue,20250613,2025-06-25 05:03:35,0,0,"171,671,844","171,671,844","171,671,844",0,0
4,salisa,TB3R000100,Broadband Revenue,20250616,2025-06-25 05:03:35,0,0,"170,318,783","170,318,783","170,318,783",0,0
5,salisa,TB3R000100,Broadband Revenue,20250619,2025-06-25 05:03:35,0,0,"165,169,215","165,169,215","165,169,215",0,0
6,salisa,TB3R000100,Broadband Revenue,20250622,2025-06-25 05:03:35,0,0,"161,747,098","161,747,098","161,747,098",0,0


In [19]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250301
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250601


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250602,2025-06-25 05:08:44,0,0,"31,977,323","31,957,585","31,956,793",0,0
1,salisa,TB4R000100,TVS Revenue,20250606,2025-06-25 05:08:44,0,0,"8,866,440","8,922,056","8,910,708",0,0
2,salisa,TB4R000100,TVS Revenue,20250610,2025-06-25 05:08:44,0,0,"26,754,193","26,840,319","26,849,834",0,0
3,salisa,TB4R000100,TVS Revenue,20250613,2025-06-25 05:08:44,0,0,"19,893,173","19,931,507","19,933,457",0,0
4,salisa,TB4R000100,TVS Revenue,20250616,2025-06-25 05:08:44,0,0,"21,342,294","21,338,612","21,332,775",0,0
5,salisa,TB4R000100,TVS Revenue,20250619,2025-06-25 05:08:44,0,0,"18,633,735","18,651,031","18,653,286",0,0
6,salisa,TB4R000100,TVS Revenue,20250622,2025-06-25 05:08:44,0,0,"18,143,336","18,143,336","18,143,554",0,0


## Issue
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [20]:
# ''' Execute Issue by case '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+ PARALLEL(8)*/ 
#          REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
#         , MAX(LOAD_DATE) LOAD_DATE
#         , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
#     FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
#     WHERE METRIC_CD = 'TB1S000600' --Prepay Active Caller Subs
#     AND EXISTS (
#         SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
#         FROM (
#             SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#         ) TMP
#         WHERE TMP_CD = AREA_CD )
#     AND TM_KEY_DAY >= {v_start_date}
#     GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Issue DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [21]:
# ''' TB1S000600 : Prepay Active Caller Subs '''

# v_metric_cd = 'TB1S000600'
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250101
# print(f'\nTB1S000600 : Prepay Active Caller Subs...\n')
# print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

# TB1S000600_day_df = chk_src_df.copy()
# TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['METRIC_CD']==v_metric_cd]
# TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
# TB1S000600_day_df = TB1S000600_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

# mod_col_list = TB1S000600_day_df.iloc[:, 5:12].columns.tolist()
# for col in mod_col_list:
#     TB1S000600_day_df[col] = TB1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# TB1S000600_day_df